In [1]:
from ast import increment_lineno
import numpy as np
import pandas as pd
import seaborn as sns 
import tensorflow as tf 

from tensorflow import keras as ks
from tensorflow.keras.layers import Dense
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

%matplotlib inline 

In [ ]:
%pip install keras-tuner

In [3]:
import keras_tuner as kt

In [ ]:
boston_ds = datasets.load_boston()
columns = boston_ds.feature_names
target = boston_ds.target
data = pd.DataFrame(boston_ds.data, columns=columns).astype(np.float32)
data.head()

In [5]:
data['Price'] = target.astype(np.float32)
target = data['Price']
# data.head()

In [ ]:
data.isnull().sum()

In [ ]:
plt.figure(figsize=(11.7,8.27))
sns.displot(data['Price'], kde=True, height=7, aspect=2, bins=30)
plt.show()

In [ ]:
corr = data.corr().round(2)
plt.figure(figsize=(12.7,8.27))
sns.heatmap(data=corr, annot=True)

In [ ]:
features = ['RM', 'LSTAT']
plt.figure(figsize=(15.7,5.27))
for i, col in enumerate(features):
    plt.subplot(1, len(features) , i+1)
    x = data[col]
    y = target
    plt.scatter(x, y, marker='o')
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel('Price')

In [6]:
X = pd.DataFrame(np.c_[data['LSTAT'], data['RM']], columns = ['LSTAT','RM'])
Y = data['Price']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=5)

In [8]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
mms.fit(X_train)
X_train = mms.transform(X_train)
X_test = mms.transform(X_test)

In [9]:
from tensorflow.keras.models import Sequential

def model_builder(hp):
  model = Sequential(name='BostonHousingPrices')
  # automate unites hp
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(Dense(units=hp_units, input_dim = 2, activation='relu', name='input'))
  for i in range(1, hp.Int("num_layers", 2, 6)):
    model.add(
        tf.keras.layers.Dense(
            units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
            activation="relu", name='Hidden-' + str(i)))
    
  model.add(Dense(1, activation='linear', name='Output'))
  # automate learning_rate hp
  lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                loss='mean_absolute_error', metrics=['mae'])
  return model

In [ ]:
tuner = kt.RandomSearch(
    hypermodel=model_builder,
    objective="val_loss",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="/content/hp_search_results/",
    project_name="TF_SLR",
)
tuner.search_space_summary()

In [ ]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2)

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete.\n
Optimal No of Layers: {best_hps.get('num_layers')}\n
Optimal Learning-Rate: {best_hps.get('learning_rate')}
""")

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)
from keras import callbacks
es = callbacks.EarlyStopping(monitor ="val_loss",
                             mode="min", patience=5,
                             restore_best_weights=True)
hist = hypermodel.fit(X_train, y_train, epochs=30, validation_split=0.2, callbacks=[es])

In [14]:
hypermodel.save("slr_model.h5")

In [15]:
import tempfile
import os

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    hypermodel,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)
print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp/1


Saved model:
total 108
drwxr-xr-x 2 root root  4096 Oct 25 08:58 assets
-rw-r--r-- 1 root root  8328 Oct 25 08:58 keras_metadata.pb
-rw-r--r-- 1 root root 89739 Oct 25 08:58 saved_model.pb
drwxr-xr-x 2 root root  4096 Oct 25 08:58 variables


In [ ]:
result = hypermodel.evaluate(X_test, y_test)
print(result)
for i in range(len(result)):
    print(f'{hypermodel.metrics_names[i]} ---> {result[i]}')

In [ ]:
prediction_data = np.array([6.0910, 10.46])
prediction_data = mms.transform(prediction_data.reshape(1, -1))
predict_result = model.predict(prediction_data)
print(f'Predicted price: {predict_result[0, 0]}')

In [16]:
import sys
# We need sudo prefix if not on a Google Colab.
if 'google.colab' not in sys.modules:
  SUDO_IF_NEEDED = 'sudo'
else:
  SUDO_IF_NEEDED = ''


In [17]:
!sudo apt-get purge tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package tensorflow-model-server


In [18]:
!wget 'http://storage.googleapis.com/tensorflow-serving-apt/pool/tensorflow-model-server-1.15.0/t/tensorflow-model-server/tensorflow-model-server_1.15.0_all.deb'
!dpkg -i tensorflow-model-server_1.15.0_all.deb

--2022-10-25 09:01:39--  http://storage.googleapis.com/tensorflow-serving-apt/pool/tensorflow-model-server-1.15.0/t/tensorflow-model-server/tensorflow-model-server_1.15.0_all.deb
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.188.48, 172.253.62.128, 142.251.45.16, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.188.48|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 142602722 (136M) [application/x-debian-package]
Saving to: ‘tensorflow-model-server_1.15.0_all.deb’

tensorflow-model-se 100%[===================>] 136.00M  12.6MB/s    in 11s     

2022-10-25 09:01:51 (12.2 MB/s) - ‘tensorflow-model-server_1.15.0_all.deb’ saved [142602722/142602722]

Selecting previously unselected package tensorflow-model-server.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack tensorflow-model-server_1.15.0_all.deb ...
Unpacking tensorflow-model-server (1.15.0) ...
Setting up tensorflow-mod

In [19]:
import os
os.environ["MODEL_DIR"] = MODEL_DIR

In [20]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501\
  --model_name=slr_model\
  --model_base_path="${MODEL_DIR}" > server.log 2>&1

In [21]:
!tail server.log

[warn] getaddrinfo: address family for nodename not supported
[evhttp_server.cc : 238] NET_LOG: Entering the event loop ...


In [22]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": [[15.923,20]]})

In [23]:
# docs_infra: no_execute
!pip install -q requests

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/slr_model:predict', data=data, headers=headers)

In [24]:
predictions = json_response.json()
print(f"Model think that the price of the house should be: {predictions['predictions'][0]}")

Model think that the price of the house should be: [161.899231]
